In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.mplot3d import Axes3D 
import matplotlib.pyplot as plt
import plotly.graph_objs as go
import plotly.express as px
from scipy.signal import savgol_filter

In [2]:
data_dir = '../out/Screwdriving Model Experiments'
observation_file = 'observation_data.csv'
raw_observation_file = 'raw_observation_data.csv'
out_file = 'smoothed_observation_data.csv'
sensor_file = 'sensor_data.csv'
merged_file = 'merged_data.csv'
subdirs = os.listdir(data_dir)
files = list(map(lambda sd: os.path.join(data_dir, sd, observation_file), subdirs))
# files

In [ ]:
intrinsics_matrix_left = np.array([
    [955.717, 0, 940.365],
    [0, 955.717, 552.377],
    [0, 0, 1]
])

def pixel_to_3d(pixel_x, pixel_y, depth, intrinsics_matrix=intrinsics_matrix_left):
    pixel_homogeneous = np.array([[pixel_x], [pixel_y], [1]])
    K_inv = np.linalg.inv(intrinsics_matrix)
    pixel_depth = np.array([[pixel_x * depth], [pixel_y * depth], [depth]])
    coordinate_3d = np.dot(K_inv, pixel_depth)

    return coordinate_3d.reshape(3,).tolist()

In [3]:
for subdir in subdirs:
    raw_observation_path = os.path.join(data_dir, subdir, raw_observation_file)
    df = pd.read_csv(raw_observation_path)
    df.interpolate(method='linear', axis=0, inplace=True)
    df.interpolate(method='backfill', axis=0, inplace=True)
    
    # df['X'] = df['X'].ewm(alpha=0.3).mean()
    # df['Y'] = df['Y'].ewm(alpha=0.3).mean()
    # df['Z'] = df['Z'].ewm(alpha=0.9).mean()
    # df['X'] = savgol_filter(df['X'], window_length=10, polyorder=4)
    # df['Y'] = savgol_filter(df['Y'], window_length=10, polyorder=4)
    # df['Z'] = savgol_filter(df['Z'], window_length=10, polyorder=4)
    

    for column in ['X', 'Y', 'Z']:
        df[column] = savgol_filter(df[column], window_length=10, polyorder=4)
        # df[column] = df[column].rolling(window=3, min_periods=1).mean()
    df = df.assign(Z=df['Z'].iloc[-20:].mean())

    out_path = os.path.join(data_dir, subdir, out_file)
    df.to_csv(out_path, index=False)

    for row in range(df.shape[0]):
        coord = pixel_to_3d(df['X'].iloc[row], df['Y'].iloc[row], df['Z'].iloc[row])
        df['X']

        
    fig = go.Figure(data=[go.Scatter3d(
        x=df['X'],
        y=df['Y'],
        z=df['Z'],
        mode='markers',
        marker=dict(
            size=1,
            opacity=0.8
        ),
    )])
    fig.update_layout(width=800, height=800)
    fig.show()
